In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml.reader.gpu.cudf_reader import CudfReader
from lightautoml.reader.base import PandasToPandasReader

from lightautoml.transformers.base import SequentialTransformer

from lightautoml.pipelines.utils import get_columns_by_role

from lightautoml.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml.transformers import numeric, categorical, datetime

from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_11204/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml.automl.base import AutoML

from lightautoml.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml.tasks import Task

from lightautoml.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml.ml_algo.boost_cb import BoostCB
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [8]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[13:22:37] Stdout logging level is INFO2.
[13:22:37] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[13:22:37] Task: multilabel

[13:22:37] Start automl preset with listed constraints:
[13:22:37] - time: 3600.00 seconds
[13:22:37] - CPU: 4 cores
[13:22:37] - memory: 16 GB

[13:22:37] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[13:22:41] Feats was rejected during automatic roles guess: []
[13:22:41] Layer 1 train process start. Time left 3596.01 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[13:22:42] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[13:22:42] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[13:22:44] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[13:22:47] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[13:22:50] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.349972511434353
[13:22:50] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[13:22:50] Time left 3587.05 secs



...

[13:23:12] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[13:23:21] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[13:23:30] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.322678681671603
[13:23:30] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[13:23:30] Time left 3547.78 secs

[13:23:30] Layer 1 training completed.

[13:23:30] Blending: optimization starts with equal weights and score -7.316078406621999
[13:23:30] Blending: iteration 0: score = -7.315880716742469, weights = [0.5771783 0.4228217]
[13:23:30] Blending: iteration 1: score = -7.315880716742469, weights = [0.5771783 0.4228217]
[13:23:30] Blending: no score update. Terminated

[13:23:30] Automl preset training completed in 52.64 seconds

[13:23:30] Model description:
Final prediction for new objects (level 0) = 
	 0.57718 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.42282 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [9]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb
[13:23:32] CatBoost uses as obj. MultiCrossEntropy.


In [10]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['pb', 'pb_tuned']]}
)

In [11]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[13:23:32] Stdout logging level is INFO2.
[13:23:32] Task: multilabel

[13:23:32] Start automl preset with listed constraints:
[13:23:32] - time: 3600.00 seconds
[13:23:32] - CPU: 1 cores
[13:23:32] - memory: 16 GB

[13:23:32] Train data shape: (14644, 22)
[13:23:34] Feats was rejected during automatic roles guess: []
[13:23:34] Layer 1 train process start. Time left 3598.01 secs
[13:23:37] Stdout logging level is INFO.
[13:23:37] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:23:39] Iter 0; Sample 0, BCE = 0.378979570680358; 
[13:23:40] Iter 100; Sample 0, BCE = 0.3534222360253217; 
[13:23:40] Iter 200; Sample 0, BCE = 0.3525946532857303; 
[13:23:41] Iter 300; Sample 0, BCE = 0.3527405704006093; 
[13:23:42] Iter 400; Sample 0, BCE = 0.35308490903773493; 
[13:23:42] Early stopping at iter 401, best iter 201, best_score 0.3525946532857303
[13:23:42] Selector_PB fitting and predicting completed
[13:23:42] Start fitting Lvl_0_Pipe_0_Mod_0_PB ...
[13:23:42] ===== Start wor

[13:24:22] Stdout logging level is INFO.
[13:24:22] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:24:22] Iter 0; Sample 0, BCE = 0.3795064078606776; 
[13:24:22] Iter 100; Sample 0, BCE = 0.35471263279677656; 
[13:24:23] Iter 200; Sample 0, BCE = 0.35285760041564196; 
[13:24:23] Iter 300; Sample 0, BCE = 0.35248664152209835; 
[13:24:24] Iter 400; Sample 0, BCE = 0.3523464415541972; 
[13:24:24] Iter 500; Sample 0, BCE = 0.35235080417536735; 
[13:24:25] Iter 600; Sample 0, BCE = 0.35243431905060324; 
[13:24:25] Early stopping at iter 684, best iter 484, best_score 0.3523198775675005
[13:24:25] Stdout logging level is INFO.
[13:24:25] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:24:25] Iter 0; Sample 0, BCE = 0.3790609169878739; 
[13:24:26] Iter 100; Sample 0, BCE = 0.35359199875102676; 
[13:24:26] Iter 200; Sample 0, BCE = 0.35243274782393164; 
[13:24:27] Iter 300; Sample 0, BCE = 0.35237677824923824; 
[13:24:27] Iter 400; Sample 0, BCE = 0.35252617462

[13:25:07] Stdout logging level is INFO.
[13:25:07] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:25:07] Iter 0; Sample 0, BCE = 0.37950639405724396; 
[13:25:07] Iter 100; Sample 0, BCE = 0.35470894073375053; 
[13:25:08] Iter 200; Sample 0, BCE = 0.3528438689381279; 
[13:25:08] Iter 300; Sample 0, BCE = 0.35248395451725895; 
[13:25:08] Iter 400; Sample 0, BCE = 0.35234366362005815; 
[13:25:09] Iter 500; Sample 0, BCE = 0.3523923110571456; 
[13:25:09] Early stopping at iter 588, best iter 388, best_score 0.3523401196636508
[13:25:10] Stdout logging level is INFO.
[13:25:10] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:25:10] Iter 0; Sample 0, BCE = 0.37860020179492965; 
[13:25:11] Iter 100; Sample 0, BCE = 0.35367425633076854; 
[13:25:11] Iter 200; Sample 0, BCE = 0.3542431828605407; 
[13:25:12] Iter 300; Sample 0, BCE = 0.3552247896970489; 
[13:25:13] Early stopping at iter 318, best iter 118, best_score 0.3536425908702388
[13:25:13] Stdout logging 

[13:25:53] Iter 100; Sample 0, BCE = 0.3547930403277224; 
[13:25:53] Iter 200; Sample 0, BCE = 0.35279570843612484; 
[13:25:53] Iter 300; Sample 0, BCE = 0.35232655469440144; 
[13:25:54] Iter 400; Sample 0, BCE = 0.3521914474059614; 
[13:25:54] Iter 500; Sample 0, BCE = 0.35223103259737126; 
[13:25:55] Iter 600; Sample 0, BCE = 0.3522474453419567; 
[13:25:55] Early stopping at iter 645, best iter 445, best_score 0.3521719369548748
[13:25:55] Stdout logging level is INFO.
[13:25:55] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:25:55] Iter 0; Sample 0, BCE = 0.37951331600691485; 
[13:25:56] Iter 100; Sample 0, BCE = 0.35484223713331314; 
[13:25:56] Iter 200; Sample 0, BCE = 0.35280759380960974; 
[13:25:57] Iter 300; Sample 0, BCE = 0.35231844777607624; 
[13:25:57] Iter 400; Sample 0, BCE = 0.35221186916962804; 
[13:25:57] Iter 500; Sample 0, BCE = 0.3522507276209971; 
[13:25:58] Iter 600; Sample 0, BCE = 0.35225935074051273; 
[13:25:58] Early stopping at iter 645, best

[13:26:37] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:26:37] Iter 0; Sample 0, BCE = 0.3795111799321959; 
[13:26:38] Iter 100; Sample 0, BCE = 0.35475709860097276; 
[13:26:38] Iter 200; Sample 0, BCE = 0.352793890321681; 
[13:26:39] Iter 300; Sample 0, BCE = 0.3523619670813461; 
[13:26:39] Iter 400; Sample 0, BCE = 0.35220797711582463; 
[13:26:39] Iter 500; Sample 0, BCE = 0.3522567513616606; 
[13:26:40] Iter 600; Sample 0, BCE = 0.35229835153514943; 
[13:26:40] Early stopping at iter 647, best iter 447, best_score 0.35219602551685536
[13:26:40] Stdout logging level is INFO.
[13:26:40] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:26:40] Iter 0; Sample 0, BCE = 0.37915619187740907; 
[13:26:41] Iter 100; Sample 0, BCE = 0.3542171945944911; 
[13:26:41] Iter 200; Sample 0, BCE = 0.3525806273166757; 
[13:26:42] Iter 300; Sample 0, BCE = 0.3523468321764155; 
[13:26:42] Iter 400; Sample 0, BCE = 0.3524065443212254; 
[13:26:43] Iter 500; Sample 0, BCE = 

[13:27:23] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:27:23] Iter 0; Sample 0, BCE = 0.3795097232309018; 
[13:27:23] Iter 100; Sample 0, BCE = 0.3547485105321631; 
[13:27:24] Iter 200; Sample 0, BCE = 0.3527876947759746; 
[13:27:24] Iter 300; Sample 0, BCE = 0.35237844772013593; 
[13:27:25] Iter 400; Sample 0, BCE = 0.35224564447951007; 
[13:27:25] Iter 500; Sample 0, BCE = 0.35232899096321146; 
[13:27:25] Iter 600; Sample 0, BCE = 0.35236101174108947; 
[13:27:25] Early stopping at iter 604, best iter 404, best_score 0.3522424467932649
[13:27:26] Stdout logging level is INFO.
[13:27:26] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:27:26] Iter 0; Sample 0, BCE = 0.379530315608852; 
[13:27:26] Iter 100; Sample 0, BCE = 0.35561822701952533; 
[13:27:27] Iter 200; Sample 0, BCE = 0.35328862329151073; 
[13:27:27] Iter 300; Sample 0, BCE = 0.35265959966023447; 
[13:27:27] Iter 400; Sample 0, BCE = 0.35241407509591355; 
[13:27:28] Iter 500; Sample 0, BCE

[13:28:09] Stdout logging level is INFO.
[13:28:09] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:28:09] Iter 0; Sample 0, BCE = 0.3795101702445473; 
[13:28:10] Iter 100; Sample 0, BCE = 0.3549982445532858; 
[13:28:10] Iter 200; Sample 0, BCE = 0.35289233613298027; 
[13:28:11] Iter 300; Sample 0, BCE = 0.3523901043055891; 
[13:28:12] Iter 400; Sample 0, BCE = 0.35218845700233453; 
[13:28:12] Iter 500; Sample 0, BCE = 0.35220035896804697; 
[13:28:13] Iter 600; Sample 0, BCE = 0.3522173872858711; 
[13:28:13] Early stopping at iter 611, best iter 411, best_score 0.3521794296347071
[13:28:13] Stdout logging level is INFO.
[13:28:13] GDBT train starts. Max iter 3000, early stopping rounds 200
[13:28:13] Iter 0; Sample 0, BCE = 0.379510790273792; 
[13:28:13] Iter 100; Sample 0, BCE = 0.3547257942788838; 
[13:28:14] Iter 200; Sample 0, BCE = 0.35273599812491524; 
[13:28:14] Iter 300; Sample 0, BCE = 0.35232708647252464; 
[13:28:15] Iter 400; Sample 0, BCE = 0.352216435247459

[13:28:51] Stdout logging level is INFO.
[13:28:51] GDBT train starts. Max iter 100, early stopping rounds 100
[13:28:51] Iter 0; Sample 0, BCE = 0.37167224740630583; 
[13:28:51] Iter 99; Sample 0, BCE = 0.3454110955823254; 
[13:28:51] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_1_Tuned_PB (orig) =====
[13:28:51] Stdout logging level is INFO.
[13:28:51] GDBT train starts. Max iter 100, early stopping rounds 100
[13:28:51] Iter 0; Sample 0, BCE = 0.37546002462908346; 
[13:28:52] Iter 99; Sample 0, BCE = 0.34903742871040344; 
[13:28:52] Fitting Lvl_0_Pipe_0_Mod_1_Tuned_PB finished. score = -7.318497874361673
[13:28:52] Lvl_0_Pipe_0_Mod_1_Tuned_PB fitting and predicting completed
[13:28:52] Time left 3280.27 secs

[13:28:52] Layer 1 training completed.

[13:28:52] Blending: Optimization starts with equal weights and score -7.315046419552101
[13:28:52] Blending, iter 0: score = -7.314921986288271, weights = [0.61351395 0.38648605]
[13:28:52] Blending, iter 1: score = -7.3149219862

In [12]:
gpu_inf = automl_gpu.predict(data)

In [13]:
automl_gpu.to_cpu()

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[13:28:55] CatBoost uses as obj. MultiCrossEntropy.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[13:28:55] CatBoost uses as obj. MultiCrossEntropy.


In [14]:
cpu_inf = automl_gpu.predict(data)

In [15]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 11204. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-12-10 13:28:58,220 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-10 13:28:58,220 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:59643': None}

In [16]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

multilabel isn`t supported in lgb
[13:28:58] CatBoost uses as obj. MultiCrossEntropy.
[13:28:58] Stdout logging level is INFO2.
[13:28:58] Task: multilabel

[13:28:58] Start automl preset with listed constraints:
[13:28:58] - time: 3600.00 seconds
[13:28:58] - CPU: 1 cores
[13:28:58] - memory: 16 GB

[13:28:58] Train data shape: (14644, 22)


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/rishat/miniconda3/envs/rapids-22.10/include/rmm/mr/device/cuda_memory_resource.hpp

Exception ignored in: 'cupy.cuda.thrust.cupy_malloc'
Traceback (most recent call last):
  File "cupy/cuda/memory.pyx", line 742, in cupy.cuda.memory.alloc
  File "/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/rmm/rmm.py", line 230, in rmm_cupy_allocator
    buf = librmm.device_buffer.DeviceBuffer(size=nbytes, stream=stream)
  File "device_buffer.pyx", line 87, in rmm._lib.device_buffer.DeviceBuffer.__cinit__
MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/rishat/miniconda3/envs/rapids-22.10/include/rmm/mr/device/cuda_memory_resource.hpp


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/rishat/miniconda3/envs/rapids-22.10/include/rmm/mr/device/cuda_memory_resource.hpp

Exception ignored in: 'cupy.cuda.thrust.cupy_malloc'
Traceback (most recent call last):
  File "cupy/cuda/memory.pyx", line 742, in cupy.cuda.memory.alloc
  File "/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/rmm/rmm.py", line 230, in rmm_cupy_allocator
    buf = librmm.device_buffer.DeviceBuffer(size=nbytes, stream=stream)
  File "device_buffer.pyx", line 87, in rmm._lib.device_buffer.DeviceBuffer.__cinit__
MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/rishat/miniconda3/envs/rapids-22.10/include/rmm/mr/device/cuda_memory_resource.hpp


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/rishat/miniconda3/envs/rapids-22.10/include/rmm/mr/device/cuda_memory_resource.hpp

In [17]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
#print(oof_pred_mgpu.data.T)

[[0.1501774  0.22397435 0.14758942 ... 0.44566536 0.13061096 0.18017948]
 [0.07287782 0.08673199 0.06682754 ... 0.01402428 0.06585819 0.0567576 ]
 [0.89008    0.8305474  0.8765009  ... 0.59638417 0.8885627  0.8492751 ]
 ...
 [0.07046289 0.15758915 0.08354587 ... 0.3767133  0.06494584 0.07676939]
 [0.1477636  0.2681366  0.16913156 ... 0.3954498  0.15625282 0.19699524]
 [0.0185611  0.03289489 0.01765765 ... 0.094668   0.01258386 0.02004383]]

[[0.1503371  0.2217514  0.1470724  ... 0.4244039  0.13016044 0.18693276]
 [0.07188706 0.08237167 0.06798995 ... 0.01360011 0.06615224 0.05822577]
 [0.8921318  0.8365458  0.88534045 ... 0.54654235 0.8924883  0.8452541 ]
 ...
 [0.05580416 0.12316084 0.06251336 ... 0.39107263 0.05276939 0.06616735]
 [0.15830034 0.28031355 0.17430052 ... 0.415084   0.16059293 0.22068253]
 [0.01759158 0.03067804 0.01674225 ... 0.07613056 0.01367549 0.02034565]]

[[0.15285486 0.23374644 0.16130635 ... 0.4569816  0.12774244 0.20456544]
 [0.06231772 0.08257582 0.07412326 ..